# Implementing Gradient and Hessian of Variance Components Vector

In this notebook we will use the dyestuff dataset from MixedModels to code the gradient and hessian of the variance component vector derived on paper.

In [1]:
using RDatasets, Test, GLM, LinearAlgebra, GLMCopula, BenchmarkTools, Revise
using LinearAlgebra: BlasReal, copytri!
# Dataframe with columns: Batch (Categorical), Yield (Int32)
dyestuff = dataset("lme4", "Dyestuff")
groups = unique(dyestuff[!, :Batch])
n, p, m = length(groups), 1, 2
d = Normal()
link = IdentityLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = dyestuff[!, :Batch] .== grp
    ni = count(gidx)
    y = Float64.(dyestuff[gidx, :Yield])
    X = ones(ni, 1)
    V = [ones(ni, ni), Matrix(I, ni, ni)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [2]:
# initialize β and τ from least square solution
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;

gcm.β = [1527.4999999999998]
gcm.τ = [0.000331645019946949]
gcm.Σ = [0.8636138778492553, 1.842649569420672e-9]


### Now lets look at pieces we need for gradient with respect to variance component vector

we need to form $b_i$ and $c_i$ vectors for each observation $i$. 


In [3]:
# hard coded should be of length m with elements r^t V[k] r / 2 k in [1, m]
b_i = zeros(m)
i = 1
for k in 1:m
    b_i[k] = 0.5 * transpose(gcm.data[i].res) * gcm.data[i].V[k] * gcm.data[i].res
end
b_i

2-element Array{Float64,1}:
 6328.124999999874
 9215.624999999975

In [4]:
# using function show what we are storing
rsstotal = 0.0
for i in eachindex(gcm.data)
    update_res!(gcm.data[i], gcm.β)
    rsstotal += abs2(norm(gcm.data[i].res))  # needed for updating τ in normal case
    standardize_res!(gcm.data[i])            # standardize the residuals GLM variance(μ)
    GLMCopula.update_quadform!(gcm.data[i]) # with standardized residuals
    gcm.QF[i, :] = gcm.data[i].q
end

@show gcm.data[1].q

(gcm.data[1]).q = [6328.124999999874, 9215.624999999975]


2-element Array{Float64,1}:
 6328.124999999874
 9215.624999999975

In [5]:
# # each row is an independent cluster i
gcm.QF

6×2 Array{Float64,2}:
  6328.12    9215.62
     3.125   2215.63
 16653.1     6215.63
 10878.1    11615.6
 65703.1    18140.6
 41328.1    10190.6

## we need $1 + a^t * b_i$ and $1 + a^t * c_i$ for the denominator

In [6]:
dot(gcm.Σ, b_i)

5465.056587745877

In [7]:
gcm.Σ[1] * b_i[1] + gcm.Σ[2] * b_i[2]

5465.056587745877

In [8]:
# first term of gradient with respect to a for i = 1 first observation
# need to sum over i at the end
gradient_term1 = inv(1 + dot(gcm.Σ, b_i)) * b_i

2-element Array{Float64,1}:
 1.1577130420103283
 1.6859732152535885

# c_i

In [9]:
# should be of length m with elements tr(V[k])/ 2 k in [1, m]
c_i = zeros(m)
i = 1
for k in 1:m
    c_i[k] = 0.5 * tr(gcm.data[i].V[k])
end
c_i

2-element Array{Float64,1}:
 2.5
 2.5

In [10]:
# second term of gradient with respect to a for i = 1
# need to sum over i at the end
gradient_term2 = -inv(1 + dot(gcm.Σ, c_i)) * c_i

2-element Array{Float64,1}:
 -0.7913809875559619
 -0.7913809875559619

In [11]:
gcm.TR[i, :]

2-element Array{Float64,1}:
 2.5
 2.5

In [12]:
gradient = gradient_term1 + gradient_term2

2-element Array{Float64,1}:
 0.36633205445436645
 0.8945922276976266

In [13]:
gcm = GLMCopulaVCModel(gcs);
# initialize β and τ from least square solution
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;


# not as computationally efficient just thinking
"""
update_Σ_newton1!(gcm)

Update Σ gradient and Hessian for Newton's Algorithm, given β.
"""
function update_Σ_newton1!(
    gcm::GLMCopulaVCModel{T, D, Link}) where {T <: BlasReal, D, Link}
    rsstotal = zero(T)
    fill!(gcm.∇Σ, 0.0)
    @inbounds for i in eachindex(gcm.data)
        update_res!(gcm.data[i], gcm.β)
        rsstotal += abs2(norm(gcm.data[i].res))  # needed for updating τ in normal case
        standardize_res!(gcm.data[i])            # standardize the residuals GLM variance(μ)
        GLMCopula.update_quadform!(gcm.data[i]) # with standardized residuals
        gcm.QF[i, :] = gcm.data[i].q
    end
    mul!(gcm.storage_n, gcm.QF, gcm.Σ)
    gcm.storage_n .= inv.(1 .+ gcm.storage_n)
    
    mul!(gcm.storage_n2, gcm.TR, gcm.Σ)
    gcm.storage_n2 .= inv.(1 .+ gcm.storage_n2) 

    @inbounds for i in eachindex(gcm.data)
        mul!(gcm.data[i].∇Σ1, gcm.storage_n[i], gcm.QF[i, :])
        mul!(gcm.data[i].∇Σ2, gcm.storage_n2[i], gcm.TR[i, :])
        gcm.data[i].∇Σ2 .*= -one(T)
        gcm.data[i].∇Σ .= gcm.data[i].∇Σ1 .+ gcm.data[i].∇Σ2
        gcm.∇Σ .+= gcm.data[i].∇Σ
    end
    return gcm.∇Σ
end

totalgradientΣ = update_Σ_newton1!(gcm)

gcm.β = [1527.4999999999998]
gcm.τ = [0.000331645019946949]
gcm.Σ = [0.8636138778492553, 1.842649569420672e-9]


2-element Array{Float64,1}:
   1.8857392414196066
 598.2236017801591

## First check if the gradient matches my hardcoded gradient

In [14]:
@test gcm.data[i].∇Σ1 == gradient_term1

Test Passed

In [15]:
@test gcm.data[i].∇Σ2 == gradient_term2

Test Passed

In [16]:
@test gcm.data[i].∇Σ == gradient

Test Passed

In [17]:
# we need to sum over i for the first gradient term

term1_n = zeros(m)
for i in 1:length(gcm.data)
    term1_n += gcm.storage_n[i] * gcm.QF[i, :]
end

term1_n

2-element Array{Float64,1}:
   6.634025166755378
 602.971887705495

In [18]:
# we need to sum over i for the second gradient term

term2_n = zeros(m)
for i in 1:length(gcm.data)
    term2_n += -gcm.storage_n2[i] * gcm.TR[i, :]
end

term2_n

2-element Array{Float64,1}:
 -4.748285925335771
 -4.748285925335771

In [19]:
@test gcm.∇Σ ≈ term1_n + term2_n

Test Passed

In [20]:
gcm = GLMCopulaVCModel(gcs);
# initialize β and τ from least square solution
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;

"""
update_Σ_newton!(gcm)

Update Σ gradient and Hessian for Newton's Algorithm, given β.
"""
function update_Σ_newton!(
    gcm::GLMCopulaVCModel{T, D, Link}) where {T <: BlasReal, D, Link}
    rsstotal = zero(T)
    fill!(gcm.∇Σ, 0.0)
    @inbounds for i in eachindex(gcm.data)
        update_res!(gcm.data[i], gcm.β)
        rsstotal += abs2(norm(gcm.data[i].res))  # needed for updating τ in normal case
        standardize_res!(gcm.data[i])            # standardize the residuals GLM variance(μ)
        GLMCopula.update_quadform!(gcm.data[i]) # with standardized residuals
        gcm.QF[i, :] = gcm.data[i].q
    end
    mul!(gcm.storage_n, gcm.QF, gcm.Σ)
    gcm.storage_n .= inv.(1 .+ gcm.storage_n)
    
    mul!(gcm.storage_n2, gcm.TR, gcm.Σ)
    gcm.storage_n2 .= inv.(1 .+ gcm.storage_n2)
    gcm.storage_n2 .*= -one(T)
    
    mul!(gcm.∇Σ1, transpose(gcm.QF), gcm.storage_n)
    mul!(gcm.∇Σ2, transpose(gcm.TR), gcm.storage_n2)
    gcm.∇Σ .+= gcm.∇Σ1
    gcm.∇Σ .+= gcm.∇Σ2
end

update_Σ_newton!(gcm)

@test gcm.∇Σ1 == term1_n
@test gcm.∇Σ2 ≈ term2_n
@test gcm.∇Σ ≈ term1_n + term2_n

gcm.β = [1527.4999999999998]
gcm.τ = [0.000331645019946949]
gcm.Σ = [0.8636138778492553, 1.842649569420672e-9]


Test Passed

### Now Benchmark the Gradient

In [21]:
@benchmark update_Σ_newton!($gcm)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.618 μs (0.00% GC)
  median time:      1.804 μs (0.00% GC)
  mean time:        1.886 μs (0.00% GC)
  maximum time:     15.617 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

## Next we can work on the Hessian

In [22]:
hessian_term1 = -inv((1 + dot(gcm.Σ, b_i))^2) * b_i * transpose(b_i)

2×2 Array{Float64,2}:
 -1.3403   -1.95187
 -1.95187  -2.84251

In [23]:
hessian_term2 = inv((1 + dot(gcm.Σ, c_i))^2) * c_i * transpose(c_i)

2×2 Array{Float64,2}:
 0.626284  0.626284
 0.626284  0.626284

In [24]:
hessian = hessian_term1 + hessian_term2

2×2 Array{Float64,2}:
 -0.714016  -1.32559
 -1.32559   -2.21622

### Not as computationally efficient just thinking in a draft

In [25]:
gcm = GLMCopulaVCModel(gcs);
# initialize β and τ from least square solution
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;


"""
update_Σ_newton1!(gcm)

Update Σ gradient and Hessian for Newton's Algorithm, given β.
"""
function update_Σ_newton2!(
    gcm::GLMCopulaVCModel{T, D, Link}) where {T <: BlasReal, D, Link}
    rsstotal = zero(T)
    rsstotal = 0.0
    fill!(gcm.∇Σ, 0.0)
    @inbounds for i in eachindex(gcm.data)
        update_res!(gcm.data[i], gcm.β)
        rsstotal += abs2(norm(gcm.data[i].res))  # needed for updating τ in normal case
        standardize_res!(gcm.data[i])            # standardize the residuals GLM variance(μ)
        GLMCopula.update_quadform!(gcm.data[i]) # with standardized residuals
        gcm.QF[i, :] = gcm.data[i].q
    end
    mul!(gcm.storage_n, gcm.QF, gcm.Σ)
    gcm.storage_n .= inv.(1 .+ gcm.storage_n)
    
    mul!(gcm.storage_n2, gcm.TR, gcm.Σ)
    gcm.storage_n2 .= inv.(1 .+ gcm.storage_n2) 

    @inbounds for i in eachindex(gcm.data)
        mul!(gcm.data[i].∇Σ1, gcm.storage_n[i], gcm.QF[i, :])
        mul!(gcm.data[i].∇Σ2, gcm.storage_n2[i], gcm.TR[i, :])
        gcm.data[i].∇Σ2 .*= -one(T)
        gcm.data[i].∇Σ .= gcm.data[i].∇Σ1 .+ gcm.data[i].∇Σ2
        gcm.∇Σ .+= gcm.data[i].∇Σ
        
        # hessian
        gcm.data[i].HΣ1 .= -gcm.data[i].∇Σ1 * gcm.storage_n[i] * transpose(gcm.QF[i, :])
        gcm.data[i].HΣ2 .= -gcm.data[i].∇Σ2 * gcm.storage_n2[i] * transpose(gcm.TR[i, :])
        gcm.data[i].HΣ .= gcm.data[i].HΣ1 .+ gcm.data[i].HΣ2
        gcm.HΣ1 .+= gcm.data[i].HΣ1
        gcm.HΣ2 .+= gcm.data[i].HΣ2
        gcm.HΣ .+= gcm.data[i].HΣ
    end
    return gcm.HΣ
end

totalhessian = update_Σ_newton2!(gcm) 
@show totalhessian
@show gcm.data[1].HΣ1
@show gcm.data[1].HΣ2
@show gcm.data[1].HΣ;

gcm.β = [1527.4999999999998]
gcm.τ = [0.000331645019946949]
gcm.Σ = [0.8636138778492553, 1.842649569420672e-9]
totalhessian = [-3.658966153259061 -506.9136917480972; -506.9136917480972 -358816.664752713]
(gcm.data[1]).HΣ1 = [-1.340299487640808 -1.951873179779166; -1.9518731797791657 -2.842505682552523]
(gcm.data[1]).HΣ2 = [0.6262838674650495 0.6262838674650495; 0.6262838674650495 0.6262838674650495]
(gcm.data[1]).HΣ = [-0.7140156201757586 -1.3255893123141165; -1.3255893123141163 -2.2162218150874735]


In [26]:
@test gcm.data[1].HΣ1 == hessian_term1

Test Passed

In [27]:
@test gcm.data[1].HΣ2 ≈ hessian_term2

Test Passed

In [28]:
@test gcm.data[1].HΣ ≈ hessian

Test Passed

In [29]:
# we need to sum over i for the first gradient term

hterm1_n = zeros(m, m)
for i in 1:length(gcm.data)
    hterm1_n += -gcm.data[i].∇Σ1 * gcm.storage_n[i] * transpose(gcm.QF[i, :])
end

hterm1_n

2×2 Array{Float64,2}:
   -7.41667  -510.671
 -510.671      -3.5882e5

In [30]:
# we need to sum over i for the second gradient term

hterm2_n = zeros(m, m)
for i in 1:length(gcm.data)
    hterm2_n += -gcm.data[i].∇Σ2 * gcm.storage_n2[i] * transpose(gcm.TR[i, :])
end

hterm2_n

2×2 Array{Float64,2}:
 3.7577  3.7577
 3.7577  3.7577

In [31]:
@test gcm.HΣ1 == hterm1_n
@test gcm.HΣ2 == hterm2_n
@test gcm.HΣ ≈ hterm1_n + hterm2_n

Test Passed

In [32]:
gcm = GLMCopulaVCModel(gcs);
# initialize β and τ from least square solution
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;

"""
update_∇Σ_HΣ!(gcm)

Update Σ gradient and Hessian for Newton's Algorithm, given β.
"""
function update_∇Σ_HΣ!(
    gcm::GLMCopulaVCModel{T, D, Link}) where {T <: BlasReal, D, Link}
    rsstotal = zero(T)
    fill!(gcm.∇Σ, 0.0)
    fill!(gcm.HΣ, 0.0)
    @inbounds for i in eachindex(gcm.data)
        update_res!(gcm.data[i], gcm.β)
        rsstotal += abs2(norm(gcm.data[i].res))  # needed for updating τ in normal case
        standardize_res!(gcm.data[i])            # standardize the residuals GLM variance(μ)
        GLMCopula.update_quadform!(gcm.data[i]) # with standardized residuals
        gcm.QF[i, :] = gcm.data[i].q
    end
    mul!(gcm.storage_n, gcm.QF, gcm.Σ)
    gcm.storage_n .= inv.(1 .+ gcm.storage_n)
    
    mul!(gcm.storage_n2, gcm.TR, gcm.Σ)
    gcm.storage_n2 .= inv.(1 .+ gcm.storage_n2)
    
    mul!(gcm.∇Σ1, transpose(gcm.QF), gcm.storage_n)
    mul!(gcm.∇Σ2, transpose(gcm.TR), gcm.storage_n2)
    gcm.∇Σ2 .*= -one(T)
    
    gcm.∇Σ .+= gcm.∇Σ1
    gcm.∇Σ .+= gcm.∇Σ2
    
    # hessian
    gcm.diagonal_n .= Diagonal(gcm.storage_n)
    mul!(gcm.hess1, transpose(gcm.QF), gcm.diagonal_n)
    
    mul!(gcm.HΣ1, gcm.hess1, transpose(gcm.hess1))
    gcm.HΣ1 .*= -one(T)
    
    gcm.diagonal_n .= Diagonal(gcm.storage_n2)
    mul!(gcm.hess2, transpose(gcm.TR), gcm.diagonal_n)
    mul!(gcm.HΣ2, gcm.hess2, transpose(gcm.hess2))
    gcm.HΣ .+= gcm.HΣ1
    gcm.HΣ .+= gcm.HΣ2
end

update_∇Σ_HΣ!(gcm)

gcm.β = [1527.4999999999998]
gcm.τ = [0.000331645019946949]
gcm.Σ = [0.8636138778492553, 1.842649569420672e-9]


2×2 Array{Float64,2}:
   -3.65897  -506.914
 -506.914      -3.58817e5

In [33]:
# make sure gradient is still right
@test gcm.∇Σ1 == term1_n
@test gcm.∇Σ2 ≈ term2_n
@test gcm.∇Σ ≈ term1_n + term2_n

Test Passed

In [34]:
@test gcm.HΣ1 == hterm1_n

Test Passed

In [35]:
@test gcm.HΣ2 == hterm2_n

Test Passed

In [36]:
@test gcm.HΣ ≈ hterm1_n + hterm2_n

Test Passed

# benchmark the function with gradient and hessian

In [37]:
@benchmark update_∇Σ_HΣ!($gcm)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.433 μs (0.00% GC)
  median time:      2.579 μs (0.00% GC)
  mean time:        2.832 μs (0.00% GC)
  maximum time:     16.330 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9